## Notes

* All segments have different lengths -> sliding window approach
  * maybe length of 100 or 200? I read that the window size matters a lot

## Open Questions

* Network architecture? -> LSTM, CNN, Hierarchical Attention?
  * Maybe implement multiple and compare?

In [41]:
import pandas as pd

In [40]:
activities = pd.read_csv("train/activities_train.csv") # Activity Labels for Segments and Nurse ID
accelerometer = pd.read_csv("train/accelerometer_train.csv") # I don't think we need it, mocap should have it covered
meditag = pd.read_csv("train/meditag_train.csv") # Just unrealistic to have IRL
mocap = pd.DataFrame()
for mf in glob.glob("train/mocap/segment*.csv"):
    mocap = mocap.append(pd.read_csv(mf))

In [49]:
mocap.head()

,2,3,4,5,6,7,8,9,10,11,...,81,82,83,84,85,86,87,88,segment_id,time_elapsed
0,191.69318,554.75897,1602.06238,241.57750,592.88843,1572.54126,101.00808,471.36523,1555.69446,269.51511,...,469.91351,88.24467,132.93138,499.82492,419.97797,98.66891,429.72110,82.23342,235,0 days 00:00:00.000000000
1,193.54549,558.02441,1601.91406,242.86856,596.21667,1572.70349,103.45854,474.15762,1555.55273,272.85904,...,482.49655,86.07478,133.97206,501.05740,419.76840,98.71282,429.83270,82.25223,235,0 days 00:00:00.010000000
2,195.46664,561.30737,1601.70361,244.85583,600.17749,1572.21509,105.86305,476.92685,1555.27222,276.23163,...,495.16177,83.77824,134.98735,502.36157,419.54572,98.79245,429.91837,82.32188,235,0 days 00:00:00.020000000
3,197.46156,564.63715,1601.43555,246.56914,603.82129,1571.96887,108.36342,479.73187,1554.97400,279.50342,...,507.82224,81.95531,136.07344,503.69806,419.30563,98.86098,430.02249,82.39525,235,0 days 00:00:00.030000000
4,199.50728,567.89349,1601.14001,248.03075,607.15277,1571.97705,110.92735,482.52811,1554.60852,282.80209,...,520.46924,80.47074,137.17154,504.94839,419.09320,98.93748,430.12210,82.45335,235,0 days 00:00:00.040000000


In [50]:
mocap.tail()

,2,3,4,5,6,7,8,9,10,11,...,81,82,83,84,85,86,87,88,segment_id,time_elapsed
5995,346.91016,1063.71448,1502.01831,416.86694,1064.21228,1471.01636,231.57780,1069.21155,1455.40295,363.99844,...,821.64319,98.55347,503.26791,1347.00378,901.50964,172.78313,1045.62219,442.67133,416,0 days 00:00:59.950000000
5996,347.95981,1062.84253,1501.70935,417.78418,1063.20996,1470.55359,232.54987,1068.64856,1455.30188,364.75443,...,821.90100,98.50520,503.22461,1346.62756,901.18579,173.48494,1045.77869,442.70737,416,0 days 00:00:59.960000000
5997,348.98941,1061.89893,1501.43042,418.75760,1062.15344,1470.19019,233.46471,1067.95020,1455.32776,365.35544,...,822.22809,98.39506,503.57016,1345.99524,901.41437,174.19606,1045.95300,442.76398,416,0 days 00:00:59.970000000
5998,349.97421,1061.01978,1501.24622,419.66510,1061.18848,1469.83362,234.39191,1067.27722,1455.43652,366.02386,...,822.55066,98.27039,504.00198,1345.78381,901.54797,174.89038,1046.14111,442.76926,416,0 days 00:00:59.980000000
5999,350.95102,1060.07568,1501.08972,420.57101,1060.21863,1469.45068,235.24417,1066.47791,1455.62463,366.18692,...,823.23315,98.36236,504.50412,1345.54614,902.02057,175.64081,1046.37805,442.75220,416,0 days 00:00:59.990000000


In [56]:
# generating windows
window_length = 200
slide = 2
windows = []
for sid in mocap.segment_id.unique():
    for start_pos in range(0, len(mocap.loc[mocap.segment_id == sid]), slide):
        end_pos = start_pos + window_length
        window = mocap.iloc[start_pos:end_pos]
        if len(window) != window_length:
            continue
        windows.append(window.to_numpy())
windows = np.array(windows)
print(windows.shape)

[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.980000000']
 [530.5615799999998 783.61176 1278.49597 ... 79.53558000000002 235
  '0 days 00:00:01.990000000']]
[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.9800000

[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.980000000']
 [530.5615799999998 783.61176 1278.49597 ... 79.53558000000002 235
  '0 days 00:00:01.990000000']]
[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.9800000

[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.980000000']
 [530.5615799999998 783.61176 1278.49597 ... 79.53558000000002 235
  '0 days 00:00:01.990000000']]
[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.9800000

[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.980000000']
 [530.5615799999998 783.61176 1278.49597 ... 79.53558000000002 235
  '0 days 00:00:01.990000000']]
[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.9800000

[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.980000000']
 [530.5615799999998 783.61176 1278.49597 ... 79.53558000000002 235
  '0 days 00:00:01.990000000']]
[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.9800000

[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.980000000']
 [530.5615799999998 783.61176 1278.49597 ... 79.53558000000002 235
  '0 days 00:00:01.990000000']]
[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.9800000

[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.980000000']
 [530.5615799999998 783.61176 1278.49597 ... 79.53558000000002 235
  '0 days 00:00:01.990000000']]
[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.9800000

[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.980000000']
 [530.5615799999998 783.61176 1278.49597 ... 79.53558000000002 235
  '0 days 00:00:01.990000000']]
[[191.69318 554.75897 1602.06238 ... 82.23342 235
  '0 days 00:00:00.000000000']
 [193.54549 558.02441 1601.91406 ... 82.25223000000003 235
  '0 days 00:00:00.010000000']
 [195.46664 561.30737 1601.70361 ... 82.32188000000002 235
  '0 days 00:00:00.020000000']
 ...
 [525.74933 785.07538 1288.68433 ... 79.77393000000002 235
  '0 days 00:00:01.970000000']
 [528.1480700000002 784.32349 1283.5263699999996 ... 79.65318 235
  '0 days 00:00:01.9800000